In [ ]:
import numpy as np
import eval_popcon as popcon

# Q2: 1D Profile of ARC

For the POPCON plots (which should have been called perfomance of plasma contours),
I adopted the code from lecture 11 (as `eval_popcon.py`).
I then modified to have a main function, that was `__name__` guarded so it would be importable.

In [ ]:
params = {
    "cfs": {
        "name": "SPARC",
        "kind": "tokamak",
        "R": 3.3,
        "a": 1.13,
        "kappa_s": 1.84,
        "S": 2.63,
        "B_tor": 9.2,
        "I_p": 7.8,
        "H_factor": 1.8,
        "M": 2.5,
        "ne" :1.3,
        "P_heat" :143,
        "rho_star": 5,
        "distance_plasma_wall": 0.1,
        "f_alpha_loss": 0.02,
        "fueling_mix": [0.5, 0.5, 5.0],
        "mu_n" : 0.5,
        "offset_n": 0.5,
        "offset_T": 0.05,
        "C_fraction": 0.02,
        "Fe_fraction": 1e-4,
        "rho_He": 5,
        "rho_prad_core": 1/2,
        "R_sync": 1,
        "P_ecrh": 25,
        "P_nbi": 0,
        "P_alpha": 100,
    },
    "type1": {
        "name": "Infinity Two",
        "kind" : "stellarator",
        "R": 12.5,
        "a_eff": 1.25,
        "H_factor": 1.2,
        "distance_plasma_wall" : 0.1,
        "B_tor" : 9.0,
        "q_23": 1.0, #???
        "ne": 2.0,
        "M": 2.5,
        "rho_star": 5
    }

}
for param_set in params.values():
    param_set["he4_dens"] = 0.05 * param_set["ne"]
    param_set["W_dens"] = 1e-6 * param_set["ne"]
    param_set["DT_dens"] = param_set["ne"] - 2*param_set["he4_dens"] - 50 * param_set["W_dens"]
    param_set["n_0"] = param_set["ne"]

## POPCORN of ARC

In [ ]:
popcon.main(params["cfs"])

## POPCON of Infinity Two

In [ ]:
popcon.main(params["type1"])

# Q2: 1D Profile Analysis for ARC

This profile was generated by an updated `predict_SPARC_performance.py` script:

![ion profile](Plots/ne_Te_profiles.png)

Let's review the actual data for how much they differ by:

In [ ]:
te = np.load("Te.npy")
ti = np.load("Ti.npy")
peak_diff = (max(te) - max(ti))
print(f"The peak difference between the ion and electron temperatures is: {peak_diff:.1f} keV.")

# Q3: Tritium Consumption for 1 GW

I assuming only $D-T$ fusion is occuring:

$$ D+ T \to n + ^4He$$

We then need the energy released per reaction (17.6 MeV) to convert this to a reaction rate. 
From there we know the consumption rate of tritium since one nucleus is consumed per reactor.

In [ ]:
POWER = 1e9 #[W]
Q_f = 17.6e6 #[eV]
J_PER_EV = 1.602e-19 #[J/eV]
N_A = 6.022e23  #[mol^-1]
TRITIUM_A = 3.0 #[amu]

In [ ]:
reaction_rate = POWER / (Q_f * J_PER_EV) #[s^-1]
reaction_rate 

In [ ]:
reaction_mol = reaction_rate / N_A #[mol/s]
tritium_rate_kg_s = reaction_mol * TRITIUM_A  / 1_000 #[kg/s]
SEC_YEAR = 60*60*24*365.2425 
tritium_rate_kg_yr = tritium_rate_kg_s * SEC_YEAR
print(f"Tritium consumption rate of a 1 GW fusion power plant is at least {tritium_rate_kg_yr:.1f} kg/yr")

# Q4: Neutron Wall Loading for ARC

The neutron wall loading is defined as:

$$ \langle N_W \rangle = \frac{P_f}{S_a} $$

With:
* $P_f$ is the fusion power
* $S_a$ is the surface area of the first wall

If we approximate SPARC as a perfect torus, it's surface area would be:

$$ S_a = 4\pi^2Rr$$

* $R$ major radius
* $r$ minor radius

In [ ]:
P_F_ARC = 525 #[MW]
R = params["cfs"]["R"]
r_p = params["cfs"]["a"]
a_eff = r_p * np.sqrt(params["cfs"]["kappa_s"])
offset = 0.10 #[m]
surface_area = 4.0 * np.pi**2 * R * (a_eff + offset) #[m^2]
wall_loading = P_F_ARC / surface_area #[MW/m^2]
print(f"The average neutron wall loading for ARC is roughly: {wall_loading:.1f} MW/m^2")

This is the published value of 2.5 MW/m^2. 

If we assume that RAFM steel is used this would lead to about 10 dpa/year per MW/m^2, or 10 $\frac{dpa-m^2}{MWyr}$. Realistically, RAFM steels are only qualified to 20 dpa, so that will be the limit.

In [ ]:
RAFM_DAMAGE_RATE = 10 #[dpa-m2/MW-year]
damage_rate = RAFM_DAMAGE_RATE * wall_loading #[dpa/year]
damage_rate

In [ ]:
DAMAGE_LIMIT = 20
time_to_limit = DAMAGE_LIMIT / damage_rate
print(f"So it would take about {time_to_limit:.1f} years to reach the material limits.")

It's apparent why CFS is investing heavily in material science, 
and the plasma chamber of SPARC has evolved significantly since this paper.